In [ ]:
import numpy as np
import torch
from torch.nn import functional as F
from tqdm import tqdm
import onnx
import onnxruntime
from onnx import numpy_helper
import os
import time

In [ ]:

def extract_onnx_weight(onnx_path):
    model = onnx.load(onnx_path)

    weights, names = [], []
    for t in model.graph.initializer:
        weights.append(numpy_helper.to_array(t))
        names.append(t.name)
        
    onnx_weight = dict()
    for name, weight in zip(names, weights):
        onnx_weight[name] = weight
    return onnx_weight

extract_onnx_weight("deit_quant_0519.onnx")
print(extract_onnx_weight("deit_quant_0519.onnx"))  

In [ ]:
import onnx
import numpy as np
from collections import defaultdict

# 加载ONNX模型
onnx_model_path = "deit_quant_0519.onnx"  # 替换为你的ONNX模型文件路径
model = onnx.load(onnx_model_path)

# 创建字典来存储节点计数
node_counts = defaultdict(int)

# 创建空数组来存储输入参数 "a" 和 "b"
input_a_array = []
input_b_array = []

# 遍历模型中的节点
for node in model.graph.node:
    if node.op_type == "QLinearMatMul":
        # 统计节点数量
        node_counts["QLinearMatMul"] += 1

        # 提取节点的输入参数 "a" 和 "b"
        input_a = node.input[0]
        input_b = node.input[3]

        # 将输入参数 "a" 和 "b" 添加到数组中
        input_a_array.append(input_a)
        input_b_array.append(input_b)

# 输出统计信息
print(f"总共找到 {node_counts['QLinearMatMul']} 个 'QLinearMatMul' 节点。")

# 使用循环输出输入参数 "a" 和 "b"
print("a=")  
values_per_line = 10  # 每行输出的值的数量
# 打印输入参数 "a" 值
print("打印输入参数 a 值")  
for i in range(len(input_a_array)):
    print(f'"{input_a_array[i]}",', end=' ')
    if (i + 1) % values_per_line == 0:
        print()  # 换行

# 打印一个换行符，以确保 "a" 和 "b" 值分开显示
print()

# 打印输入参数 "b" 值
print("打印输入参数 b 值")  
for i in range(len(input_b_array)):
    print(f'"{input_b_array[i]}",', end=' ')
    if (i + 1) % values_per_line == 0:
        print()  # 换行


In [ ]:



select_weight ="patch_embed.proj.weight_quantized" 
ONNX_PATH = "deit_quant_0519.onnx"
deit_weight = extract_onnx_weight(ONNX_PATH)
#weight = torch.from_numpy(deit_weight[select_weight])
print(type(deit_weight))
for key in deit_weight.keys():
    if key.endswith("_quantized"): 
        print(key)
       
#with open('data.txt','w') as f:    #设置文件对象
#    f.write(str(deit_weight)) #将字符串写入文件中
    
